This demo is based on Rogers' pull [here.](https://analyst.lightcast.io/analyst/?t=4b8JV)

All occupations at a 5 digit level. All years.
MSA 19100 for Dallas-Fort Worth-Arlington, TX.

Columns:
- Jobs
- Openings
- Replacements

First we need to import this library and instantiate an instance of the Lightcast API (replace user and pass with your credentials).


In [2]:
from pyghtcast.lightcast import Lightcast
lc = Lightcast(user, pass)

The Lightcast API expects a POST request of a JSON that contains "metrics" and "constraints". Metrics are just the column names. Constraints are how the data is to be filtered. The lightcast library helps us abstract by creating the JSON and POSTing it for us. All we need to do is define a list of columns we want, and a list of constraints.

Let's start with the columns. I want my query to return the values for the columns listed at the beginning of this document, for all of the years available in lightcast (2001 - 2034).

The data as it's organized in the API separates each year into it's own column for each data point, formatted like "Column.Year". For example, if I wanted to get "Jobs" in 2001, the column name is "Jobs.2001".

Let's assemble our list of columns to request.

In [3]:
start_year = 2001
end_year = 2034

col_names = [
    "Jobs",
    "Openings",
    "Replacements",
]

cols = []

# To make this easier instead of typing out every year, I'll just use a loop to append the years.
for year in range(start_year, end_year ):
    for col in col_names:
        cols.append(f"{col}.{year}")

cols

['Jobs.2001',
 'Openings.2001',
 'Replacements.2001',
 'Jobs.2002',
 'Openings.2002',
 'Replacements.2002',
 'Jobs.2003',
 'Openings.2003',
 'Replacements.2003',
 'Jobs.2004',
 'Openings.2004',
 'Replacements.2004',
 'Jobs.2005',
 'Openings.2005',
 'Replacements.2005',
 'Jobs.2006',
 'Openings.2006',
 'Replacements.2006',
 'Jobs.2007',
 'Openings.2007',
 'Replacements.2007',
 'Jobs.2008',
 'Openings.2008',
 'Replacements.2008',
 'Jobs.2009',
 'Openings.2009',
 'Replacements.2009',
 'Jobs.2010',
 'Openings.2010',
 'Replacements.2010',
 'Jobs.2011',
 'Openings.2011',
 'Replacements.2011',
 'Jobs.2012',
 'Openings.2012',
 'Replacements.2012',
 'Jobs.2013',
 'Openings.2013',
 'Replacements.2013',
 'Jobs.2014',
 'Openings.2014',
 'Replacements.2014',
 'Jobs.2015',
 'Openings.2015',
 'Replacements.2015',
 'Jobs.2016',
 'Openings.2016',
 'Replacements.2016',
 'Jobs.2017',
 'Openings.2017',
 'Replacements.2017',
 'Jobs.2018',
 'Openings.2018',
 'Replacements.2018',
 'Jobs.2019',
 'Openings.201

Constraints are slightly more complicated. We need to construct a list where each item in the list is a dictionary that describes how the data should be filtered. Each dictonary requires a a `dimensionName` field, and either a `map` or `mapLevel` field.

Dimensions are the various fields on which the API can be filtered, and can be discovered per dataset by hitting the data discovery API (TODO: elaborate on this in a separate doc).

The `map` property defines what specific values are being filtered for. `mapLevel` is a shorthand for when there is a bunch of values we want to map. We'll see examples of each here.

The [lightcast docs](https://docs.lightcast.dev/apis/core-lmi#data-queries) go more in depth on structuring queries.

Let's create the constrants list and start by defining the constraint for the geography. In this case, we want to pull data in the Dallas-Fort Worth-Arlington area, so lets add the constraint for that:

In [4]:
constraints = []

constraints.append({
    "dimensionName": "Area",
    "map": {
        "Dallas-Fort Worth-Arlington, TX": ["MSA19100"]
    },
})

The "Area" dimension handles anything related to geographies. In this case we know which MSA we want to filter on, so we can just put tho MSA in. The key values for the map are arbitrary, I decided to make it "Dallas-Fort Worth-Arlington, TX" for readability.

Now we want to pull occupations at a 5 digit level:

In [5]:
constraints.append({
    "dimensionName": "Occupation",
    "mapLevel": {"level": 5, "predicate": ["00-0000"]}
})

The dimension here is "Occupation", which allows us to filter on SOC codes. We use a mapLevel this time, because listing every single SOC code would be tedious and impractical. mapLevel lets us quickly define the spectrum of data we want.

The predicate can be thought of as the "parent" for which we should pull all its children. "00-0000" is the code for all occupations we wanted to narrow it to, for example, Management Occupations, we could do "11-0000" instead.

Now that we have our constraints, we can build our query.

In [6]:
query = lightcast.build_query_corelmi(cols, constraints)

query

{'metrics': [{'name': 'Jobs.2001'},
  {'name': 'Openings.2001'},
  {'name': 'Replacements.2001'},
  {'name': 'Jobs.2002'},
  {'name': 'Openings.2002'},
  {'name': 'Replacements.2002'},
  {'name': 'Jobs.2003'},
  {'name': 'Openings.2003'},
  {'name': 'Replacements.2003'},
  {'name': 'Jobs.2004'},
  {'name': 'Openings.2004'},
  {'name': 'Replacements.2004'},
  {'name': 'Jobs.2005'},
  {'name': 'Openings.2005'},
  {'name': 'Replacements.2005'},
  {'name': 'Jobs.2006'},
  {'name': 'Openings.2006'},
  {'name': 'Replacements.2006'},
  {'name': 'Jobs.2007'},
  {'name': 'Openings.2007'},
  {'name': 'Replacements.2007'},
  {'name': 'Jobs.2008'},
  {'name': 'Openings.2008'},
  {'name': 'Replacements.2008'},
  {'name': 'Jobs.2009'},
  {'name': 'Openings.2009'},
  {'name': 'Replacements.2009'},
  {'name': 'Jobs.2010'},
  {'name': 'Openings.2010'},
  {'name': 'Replacements.2010'},
  {'name': 'Jobs.2011'},
  {'name': 'Openings.2011'},
  {'name': 'Replacements.2011'},
  {'name': 'Jobs.2012'},
  {'nam

The `build_query_corelmi` function constructs the JSON to be POSTed to the API. I demonstrated the output of that just to show how the sausage is made, but you won't have to interact with the JSON or API directry, it's all abstracted in the following function:

In [7]:

df = lightcast.query_corelmi('emsi.us.occupation', query)

df

,Area,Occupation,Jobs.2001,Openings.2001,Replacements.2001,Jobs.2002,Openings.2002,Replacements.2002,Jobs.2003,Openings.2003,...,Replacements.2030,Jobs.2031,Openings.2031,Replacements.2031,Jobs.2032,Openings.2032,Replacements.2032,Jobs.2033,Openings.2033,Replacements.2033
0,"Dallas-Fort Worth-Arlington, TX",11-1011,9018.095654,698.348972,595.194313,8843.357700,1038.829689,583.661608,9046.858183,838.099657,...,1893.000897,29336.228198,2544.536055,1936.191061,29944.563259,2540.088389,1976.341175,30508.302598,2539.545775,2013.547971
1,"Dallas-Fort Worth-Arlington, TX",11-1021,47028.435452,4122.239825,3715.246401,46561.402255,4517.956374,3678.350778,47265.726935,5021.502312,...,12054.862846,154152.343390,13581.069350,12178.035128,155555.327419,13522.607729,12288.870866,156788.715094,13479.537930,12386.308492
2,"Dallas-Fort Worth-Arlington, TX",11-1031,251.406783,26.238433,18.101288,254.319931,19.669527,18.311035,250.547165,20.575083,...,30.056246,424.330956,37.153531,30.551829,430.927037,37.367727,31.026747,437.263244,37.581246,31.482954
3,"Dallas-Fort Worth-Arlington, TX",11-2011,1148.206144,152.418890,99.893935,1142.861484,205.525289,99.428949,1191.273163,109.273548,...,153.029482,1777.853832,172.302763,154.673283,1795.368940,172.293758,156.197098,1811.367608,171.647811,157.588982
4,"Dallas-Fort Worth-Arlington, TX",11-2021,5128.313437,455.787249,400.008448,4998.532250,515.951814,389.885516,5029.295890,495.019726,...,1534.479268,20006.057576,1866.319212,1560.472491,20311.904296,1865.146091,1584.328535,20592.721852,1863.508498,1606.232304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,"Dallas-Fort Worth-Arlington, TX",53-7081,1875.814624,337.130055,228.849384,1881.758679,270.131464,229.574559,1844.072183,280.490312,...,617.149569,5124.679643,683.938967,625.210916,5181.722582,681.522643,632.170155,5228.963573,680.855623,637.933556
794,"Dallas-Fort Worth-Arlington, TX",53-7121,584.304585,67.823406,67.779332,534.546580,69.126916,62.007403,497.238806,77.010336,...,11.550907,101.186765,14.122842,11.737665,103.571943,13.251697,12.014345,104.804102,13.011831,12.157276
795,"Dallas-Fort Worth-Arlington, TX",53-7199,478.365401,60.594228,55.490387,444.462927,56.328566,51.557700,408.282583,54.992191,...,88.291228,783.633158,112.199974,90.901446,804.931685,114.514874,93.372075,826.074484,114.891973,95.824640
796,"Dallas-Fort Worth-Arlington, TX",55-9999,7447.886306,1003.268391,811.819607,7626.540682,963.369243,831.292934,7758.161432,890.263296,...,1071.491864,9938.374974,1190.661953,1083.282872,10045.754055,1201.641758,1094.987192,10152.408621,1212.607178,1106.612540


The `query_corelmi` function will actually hit the API, and format the data it returns as a pandas DataFrame. From here you can use all of the typical pandas fuctionality to do what you need to with the data.